In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = (SparkConf().setAppName('Transform_Method').setMaster('yarn') 
    .set('spark.sql.adaptive.enabled', False)  )
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print('app_id'.ljust(40), spark.sparkContext.applicationId)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 19:50:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/18 19:50:50 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/03/18 19:50:50 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


app_id                                   application_1727681258360_0139


In [3]:
from pyspark.sql import Window
from pyspark.sql.functions import when, dense_rank, avg, col, desc, lit

def fair_avg(df, col_gr, col_val, n):
    col_gr = df[col_gr] if isinstance(col_gr, str) else col_gr
    col_val = df[col_val] if isinstance(col_val, str) else col_val
    col_val_name = col_val._jc.toString()

    wind_asc = Window.partitionBy(col_gr).orderBy(col_val)
    wind_desc = Window.partitionBy(col_gr).orderBy(col_val.desc())
    df = df\
        .withColumn('rnk_asc', dense_rank().over(wind_asc))\
        .withColumn('rnk_dsc', dense_rank().over(wind_desc))\
        .withColumn(col_val_name, \
                    when((col('rnk_asc')>n) & (col('rnk_dsc')>n), col_val)\
                         .otherwise(lit(None)))\
        .drop(col('rnk_asc'), col('rnk_dsc'))

    return df.groupBy(col_gr).agg(
                               avg(col_val_name).alias('avg_'+col_val_name)
                                 )

In [4]:
data = [('client1', 20), ('client1', 10), ('client1', 40), 
        ('client1', 20), ('client1', 50), ('client1', 40),
        ('client1', 50), ('client1', 60), ('client1', 30),

        ('client2', 20), ('client2', 50), ('client2', 40), 
        ('client2', 40), ('client2', 10), ('client2', 30),
        ('client2', 50), ('client2', 60), ('client2', 10)]

df_data = spark.createDataFrame(data, 'id: string, number: long')

In [5]:
df_data.transform(fair_avg, 'id', 'number', 0).show()
df_data.transform(fair_avg, 'id', 'number', 1).show()
df_data.transform(fair_avg, 'id', 'number', 2).show()

+-------+-----------------+
|     id|       avg_number|
+-------+-----------------+
|client2|34.44444444444444|
|client1|35.55555555555556|
+-------+-----------------+



+-------+------------------+
|     id|        avg_number|
+-------+------------------+
|client2|38.333333333333336|
|client1|35.714285714285715|
+-------+------------------+



+-------+------------------+
|     id|        avg_number|
+-------+------------------+
|client2|36.666666666666664|
|client1|36.666666666666664|
+-------+------------------+



In [8]:
def exclude_vat(df_sales, country_code_col, val_col, pc=0.15):
    #... 
    return df_sales

def with_cashback(df_sales, mcc_code):
    #... 
    return df_sales
    
def in_promo(df_sales, cmpgn_id):
    #... 
    return df_sales
    
df_res = df_data.transform(exclude_vat, 'country_code', 'amt')\
                 .transform(with_cashback, '4509')\
                 .transform(in_promo, 2047)

In [ ]:
spark.stop()